In [7]:
# Constants
from config.execution_parameters import *

# Project libraries
from src.simulator_package.simulator_functions import *
import src.hmm_package.generate_hmm
from src.hmm_package.generate_hmm import *
from src.plot_and_print_info.plots_and_print_info import *

import importlib

In [8]:
importlib.reload(src.hmm_package.generate_hmm)
from src.hmm_package.generate_hmm import *

In [2]:
advertising_campaign = SimulationClass(ANALYSIS_MODE)
advertising_campaign.simulate()

adstock = compute_adstock(observation=advertising_campaign.results["user_expositions"])
output = np.append( np.zeros([adstock.shape[0], 1]), advertising_campaign.results["user_outcome"], axis=1 )

2022-02-20 15:31:34.813993: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model = build_hmm_to_fit( states_observable=STATES_ARE_OBSERVABLE )

compiler = CompilerInfo(LR_EXPONENTIAL_DECAY)
model.compile(
    loss = compiler.loss,
    optimizer = compiler.optimizer,
    run_eagerly = True
)

fit_model(model, adstock, output)

Epoch 1/100
50/50 [==============================] - 136s 3s/step - loss: 2198.3650
Epoch 2/100
50/50 [==============================] - 142s 3s/step - loss: 1838.1738
Epoch 3/100
50/50 [==============================] - 138s 3s/step - loss: 1563.2581
Epoch 4/100
50/50 [==============================] - 140s 3s/step - loss: 1354.0989
Epoch 5/100
50/50 [==============================] - 132s 3s/step - loss: 1194.4585
Epoch 6/100
50/50 [==============================] - 129s 3s/step - loss: 1071.0237
Epoch 7/100
50/50 [==============================] - 128s 3s/step - loss: 974.6693
Epoch 8/100
50/50 [==============================] - 128s 3s/step - loss: 898.1198
Epoch 9/100
50/50 [==============================] - 128s 3s/step - loss: 836.9387
Epoch 10/100
50/50 [==============================] - 128s 3s/step - loss: 786.9731
Epoch 11/100
50/50 [==============================] - 128s 3s/step - loss: 745.7256
Epoch 12/100
50/50 [==============================] - 128s 3s/step - loss: 711.

KeyboardInterrupt: 

In [10]:
model.weights

[<tf.Variable 'transition_prob_layer_1/mu:0' shape=(9,) dtype=float32, numpy=
 array([-0.        , -0.        ,  0.67629737, -0.        , -0.        ,
         1.1280154 , -0.        , -0.        ,  0.7126047 ], dtype=float32)>,
 <tf.Variable 'transition_prob_layer_1/beta:0' shape=(27,) dtype=float32, numpy=
 array([ 0.24274294,  0.67213833, -0.18930154,  0.42957658,  0.3066869 ,
        -0.43163645,  0.908288  ,  0.7112853 , -0.17593619,  0.50711656,
         0.2704602 , -0.4840604 ,  0.9838398 ,  0.69235796, -0.55837584,
         0.6394163 ,  0.5340635 , -0.44833377,  0.49764404,  0.46627653,
        -0.2709025 ,  0.8118112 ,  0.2708874 , -0.30349028,  0.6769002 ,
         0.3169457 , -0.69520277], dtype=float32)>]

In [11]:
make_transition_matrix(-model.weights[0], model.weights[1], adstock[0:1,:,:])

<tf.Tensor: shape=(1, 30, 4, 4), dtype=float32, numpy=
array([[[[6.91500902e-02, 2.37846464e-01, 6.80722833e-01,
          1.22805610e-02],
         [6.15366518e-01, 6.34117126e-02, 3.18005085e-01,
          3.21667455e-03],
         [6.49615347e-01, 3.23765516e-01, 2.24680901e-02,
          4.15101647e-03],
         [1.00000001e-10, 1.00000001e-10, 1.00000001e-10,
          1.00000000e+00]],

        [[9.86528397e-02, 2.65040815e-01, 6.14683747e-01,
          2.16225795e-02],
         [5.66898763e-01, 9.20310020e-02, 3.34307134e-01,
          6.76310109e-03],
         [6.03907883e-01, 3.45963329e-01, 4.09354568e-02,
          9.19333473e-03],
         [1.00000001e-10, 1.00000001e-10, 1.00000001e-10,
          1.00000000e+00]],

        [[1.28442049e-01, 2.83184677e-01, 5.55061042e-01,
          3.33121791e-02],
         [5.23006022e-01, 1.22137606e-01, 3.42782646e-01,
          1.20737134e-02],
         [5.59597135e-01, 3.58362883e-01, 6.49798512e-02,
          1.70600936e-02],
      